In [0]:
#Importing spark sessions
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *


# Creating a spark session
spark = SparkSession.builder.appName("Read").getOrCreate()

In [0]:
# create schema 

my_schema = StructType([
    StructField("transaction_id", IntegerType(), True),  
    StructField("customer_id", IntegerType(), True),
    StructField("product", StringType(), True),
    StructField("quantity", IntegerType(), True),
    StructField("price", FloatType(), True),
    StructField("date", DateType(), True),
    StructField("region", StringType(), True)  
])


In [0]:
# reading data
df1 = spark.read.format("csv").option("inferSchema", True).option("header", "true").load("dbfs:/FileStore/shared_uploads/timilsina.ra@northeastern.edu/dataset-1.csv")
df1.display()

transaction_id,customer_id,product,quantity,price,date,region
1,101,Notebook,2,500,2024-01-15,North
2,102,Pencil,10,20,2024-01-16,South
3,103,Notebook,1,500,2024-01-17,East
4,104,Pen,5,100,2024-01-18,West
5,105,Notebook,3,500,2024-01-19,North
6,101,Pencil,4,20,2024-01-20,South
7,103,Pen,2,100,2024-01-21,East
8,102,Notebook,1,500,2024-01-22,West
9,104,Pencil,8,20,2024-01-23,North
10,105,Notebook,2,500,2024-01-24,South


Question 1 - Find the total sales amount for each region.

In [0]:
#Find the total sales amount for each region.
#Creating a new column for total sales 

df1 = df1.withColumn("total_sales", col("quantity") * col("price"))
df1.display()


transaction_id,customer_id,product,quantity,price,date,region,total_sales
1,101,Notebook,2,500,2024-01-15,North,1000
2,102,Pencil,10,20,2024-01-16,South,200
3,103,Notebook,1,500,2024-01-17,East,500
4,104,Pen,5,100,2024-01-18,West,500
5,105,Notebook,3,500,2024-01-19,North,1500
6,101,Pencil,4,20,2024-01-20,South,80
7,103,Pen,2,100,2024-01-21,East,200
8,102,Notebook,1,500,2024-01-22,West,500
9,104,Pencil,8,20,2024-01-23,North,160
10,105,Notebook,2,500,2024-01-24,South,1000


In [0]:
#Find the total sales amount for each region.

df1_sales_by_region = df1.groupBy("region").agg(
    sum("total_sales").alias("total_sales_by_region")
)
df1_sales_by_region.display()

region,total_sales_by_region
South,1280
East,700
West,1000
North,2660


Question 2 -Identify the most purchased product by each customer.

In [0]:
# Identify the most purchased product by each customer.
df1_total_purchased_by_customer = df1.groupBy("customer_id", "product").agg(sum("quantity").alias("total_quantity"))
df1_total_purchased_by_customer.display()


from pyspark.sql.window import *

df_window = Window.partitionBy("customer_id").orderBy(desc("total_quantity"))

df1_most_purchased = df1_total_purchased_by_customer.withColumn("rank", row_number().over(df_window))
df_most_purchased_product = df1_most_purchased.filter(col("rank") == 1).drop("rank")

df_most_purchased_product.display()


customer_id,product,total_quantity
101,Notebook,2
104,Pencil,8
105,Notebook,5
103,Pen,2
102,Pencil,10
103,Notebook,1
104,Pen,5
102,Notebook,1
101,Pencil,4


customer_id,product,total_quantity
101,Pencil,4
102,Pencil,10
103,Pen,2
104,Pencil,8
105,Notebook,5


In [0]:
#create a temp view 
df1.createOrReplaceTempView("temp_table")

In [0]:
spark.sql("SELECT * FROM temp_table").display()


transaction_id,customer_id,product,quantity,price,date,region,total_sales
1,101,Notebook,2,500,2024-01-15,North,1000
2,102,Pencil,10,20,2024-01-16,South,200
3,103,Notebook,1,500,2024-01-17,East,500
4,104,Pen,5,100,2024-01-18,West,500
5,105,Notebook,3,500,2024-01-19,North,1500
6,101,Pencil,4,20,2024-01-20,South,80
7,103,Pen,2,100,2024-01-21,East,200
8,102,Notebook,1,500,2024-01-22,West,500
9,104,Pencil,8,20,2024-01-23,North,160
10,105,Notebook,2,500,2024-01-24,South,1000


In [0]:
df_most_purchased_product = spark.sql("""
    WITH customerItemCounts AS (
        SELECT customer_id, product, SUM(quantity) AS total_quantity
        FROM temp_table
        GROUP BY customer_id, product
    ),
    ranked_item AS (
        SELECT customer_id, product, total_quantity,
               ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY total_quantity DESC) AS rank
        FROM customerItemCounts
    )
    SELECT customer_id, product, total_quantity
    FROM ranked_item
    WHERE rank = 1
""")

# Display the result
df_most_purchased_product.display()



customer_id,product,total_quantity
101,Pencil,4
102,Pencil,10
103,Pen,2
104,Pencil,8
105,Notebook,5


Question 3 - Calculate the total revenue generated by each product.

In [0]:
#Calculate the total revenue generated by each product.
 
df_total_rev_by_product = df1.groupBy("product").agg(sum("total_sales").alias("total_sales_by_product"))
df_total_rev_by_product.display()


product,total_sales_by_product
Pen,700
Notebook,4500
Pencil,440


Question 4-  Find the region with the highest total sales.

In [0]:
# Find the region with the highest total sales

df_region_total_sales = df1.groupBy("region").agg(sum("total_sales").alias("total_sales_by_region"))

df_region_total_sales.display()

# Region with the highest total sales
df_region_with_highest_sale = df_region_total_sales.orderBy(desc("total_sales_by_region")).limit(1)
df_region_with_highest_sale.display()





region,total_sales_by_region
South,1280
East,700
West,1000
North,2660


region,total_sales_by_region
North,2660


Question 5- Find customers who purchased more than 10 units of any product.

In [0]:
#Find customers who purchased more than 10 units of any product.

df_high_value = df1.groupBy("customer_id").agg(sum("quantity").alias ("total_purchase"))
df_high_value_customers = df_high_value.filter(df_high_value.total_purchase>10)
df_high_value_customers.display()

customer_id,total_purchase
102,11
104,13


Question 6- Calculate average price per product across all regions.

In [0]:
#Calculate average price per product across all regions.
df_avg_price= df1.groupBy("product").agg(avg("price").alias("Avg_Price"))
df_avg_price.display()

product,Avg_Price
Pen,100.0
Notebook,500.0
Pencil,20.0


Question 7- Rank the products by total sales in descending order.

In [0]:

# Ranking the products by total sales in descending order 
df_total_sales = df1.groupBy("product").agg(sum("total_sales").alias("total_sales"))

window_spec = Window.orderBy(desc("total_sales"))

df_ranked_products = df_total_sales.withColumn("rank", rank().over(window_spec))

df_ranked_products.display()


product,total_sales,rank
Notebook,4500,1
Pen,700,2
Pencil,440,3


Question 8- Count the number of distinct customers in each region.

In [0]:
#Count the number of distinct customers in each region.
df_distinct_customer= df1.groupBy("region").agg(countDistinct("customer_id").alias("distinct_customers"))
df_distinct_customer.display()

region,distinct_customers
South,3
East,1
West,2
North,3


Question 9- Determine which region has the highest average quantity sold per transaction.

In [0]:
#Determine which region has the highest average quantity sold per transaction.
# Average quantity sold by region 
df_highest_average = df1.groupBy("region").agg(avg("quantity").alias("avg_quantity_per_transaction"))
df_highest_average.display()

# Region with highest average 
df_highest_average_region = df_highest_average.orderBy (desc("avg_quantity_per_transaction")). limit(1)
df_highest_average_region.display()




region,avg_quantity_per_transaction
South,5.333333333333333
East,1.5
West,3.0
North,4.333333333333333


region,avg_quantity_per_transaction
South,5.333333333333333


Question 10 - Find transactions that contributed to the top 10% of total revenue.
 

In [0]:

# Compute the 90th percentile (top 10% threshold) of total_sales
threshold = df1.approxQuantile("total_sales", [0.9], 0.01)[0]

# Filter transactions contributing to the top 10% of total_sales
top_10_percent_transactions = df1.filter(col("total_sales") >= threshold)

# Display the result
top_10_percent_transactions.display()


transaction_id,customer_id,product,quantity,price,date,region,total_sales
1,101,Notebook,2,500,2024-01-15,North,1000
5,105,Notebook,3,500,2024-01-19,North,1500
10,105,Notebook,2,500,2024-01-24,South,1000
